In [19]:
import numpy as np
import scipy.stats as spst

# 1. Price an American call option with spot stock price $S_0$ = 100.0, strike $K$ = 100.0, time to maturity $T$ = 1.0 year, risk free interest rate $r$ = 6%, continuous dividend yield $q$ = 6%, volatility $\sigma$ = 35% using a 100 step CRR binomial tree. 

In [20]:
S0 = 100
K = 100
T = 1
r = 0.06
q = 0.06
sigma = 0.35 
option_type = 'call'

In [21]:
n_steps = 100

In [22]:
# black scholes model
d1 = (np.log(S0/K) + (r - q + sigma**2/2) * T) / (sigma * np.sqrt(T))
d2 = d1 - sigma * np.sqrt(T)

C = S0 * np.exp(-q * T) * spst.norm.cdf(d1) - K * np.exp(-r * T) * spst.norm.cdf(d2)

print('BSM price = %.2f' % C)

BSM price = 13.08


In [23]:
def option_payoff(S, K, option_type):
    payoff = 0.0
    
    if option_type == "call":
        payoff = np.maximum(S - K, 0)
    elif option_type == "put":
        payoff = np.maximum(K - S, 0)
    return payoff

In [24]:
def get_american_call(S0, K, T, r, q, sigma):
    dt = T/n_steps
    sqrt_dt = np.sqrt(dt)

    # CRR model parameters
    u = np.exp(sigma*sqrt_dt)
    d = 1/u
    p = (np.exp((r - q) * dt) - d)/(u - d)

    # simulate stock prices
    St = np.zeros((n_steps+1, n_steps+1))
    for i in range(n_steps+1):
        for j in range(i+1):
            St[j,i] = S0 * (u**(i-j)) * (d**(j))

    # get payoffs
    payoffs = np.zeros((n_steps+1, n_steps+1))
    payoffs[:, n_steps] = option_payoff(St[:, n_steps], K, 'call')

    # backward induction
    for j in range(n_steps-1, -1, -1):
        for i in range(j+1):
            payoffs[i,j] = np.exp(-r*dt)*(p*payoffs[i,j+1] + (1-p)*payoffs[i+1,j+1]) # never exercise early
            payoffs[i,j] = max(payoffs[i,j], St[i,j]-K)

    return payoffs, St


In [28]:
payoffs, St = get_american_call(S0, K, T, r, q, sigma)
dt = T/n_steps

delta = (payoffs[0,1] - payoffs[1,1])/(St[0,1] - St[1,1])
delta1 = (payoffs[1,2] - payoffs[2,2])/(St[1,2] - St[2,2])
delta2 = (payoffs[0,2] - payoffs[1,2])/(St[0,2] - St[1,2])

gamma = (delta2 - delta1)/(St[0,1] - St[1,1])

theta = (payoffs[1,2] - payoffs[0,0])/(2*dt)

vega = (get_american_call(S0, K, T, r, q, sigma+0.0001)[0][0,0] - payoffs[0,0])/0.0001
rho = (get_american_call(S0, K, T, r+0.0001, q, sigma)[0][0,0] - payoffs[0,0])/0.0001

print('call price = %.4f' % payoffs[0,0])
print('delta = %.4f' % delta)
print('gamma = %.4f' % gamma)
print('theta = %.4f' % theta)
print('vega = %.4f' % vega)
print('rho = %.4f' % rho)

call price = 13.2413
delta = 0.5481
gamma = 0.0112
theta = -6.0564
vega = 37.4546
rho = 33.9585


In [27]:
def american_call(S0, K, T, r, q, sigma, n_steps):
    dt = T/n_steps
    u = np.exp(sigma*np.sqrt(dt))
    d = 1/u
    p = (np.exp((r-q)*dt)-d)/(u-d)
    disc = np.exp(-r*dt)
    S = np.zeros([n_steps+1, n_steps+1])
    
    for i in range(n_steps+1):
        for j in range(i+1):
            S[j,i] = S0*(d**j)*(u**(i-j))
            
    # option
    C = np.zeros([n_steps+1, n_steps+1])
    C[:, n_steps] = np.maximum(np.zeros(n_steps+1),S[:, n_steps]-K)
    
    for i in np.arange(n_steps-1, -1, -1):
        for j in np.arange(0, i+1):
            C[j,i] = disc*(p*C[j,i+1] + (1-p)*C[j+1, i+1])
            C[j,i] = max(C[j,i], S[j, i]-K)
    delta = (C[0,1] - C[1,1])/(S[0,1] - S[1,1])
    delta1 = (C[1,2] - C[2,2])/(S[1,2]-S[2,2])
    delta2 = (C[0,2] - C[1,2])/(S[0,2] - S[1,2])
    gamma = (delta2-delta1)/(S[0,1] - S[1,1])
    theta = (C[1,2]-C[0,0]) / (2*dt)
    
    return C[0,0], delta, gamma, theta
    
american_call(100.0, 100.0, 1.0, 0.06, 0.06, 0.35, 100)

(13.241257227890095,
 0.5480841683241352,
 0.01118472156795883,
 -6.056389015633634)